## 🚦 การสร้างคอลัมน์เงื่อนไขด้วย `np.where`
---

ในงานจัดการข้อมูล เรามักจะต้องสร้างคอลัมน์ใหม่ที่ระบุสถานะตามเงื่อนไข (เช่น ถ้าวันเหลือน้อยกว่า 0 ให้แสดงว่า "หมดอายุ") เราจะใช้คำสั่ง `np.where` จาก Library **NumPy** ซึ่งทำงานได้รวดเร็วมากครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Conditional Logic** | คอน-ดิ-ชัน-นอล ลอ-จิก | **ตรรกะแบบมีเงื่อนไข** (ถ้า...จริง ให้ทำ A, ถ้าไม่จริง ให้ทำ B) |
| **Vectorized Operation** | เวก-เตอ-ไรซ์ ออป-เปอ-เร-ชัน | การคำนวณที่ทำกับข้อมูลทั้งชุดพร้อมกัน (ไม่ต้องวนลูปเอง) |
| **Binary Choice** | ไบ-นา-รี ช้อยส์ | การเลือกทางใดทางหนึ่งจาก 2 ทางเลือก |
| **Status Label** | สเต-ตัส เล-เบิล | ป้ายระบุสถานะ (เช่น "ปกติ", "เตือน", "ยกเลิก") |

### 💡 รูปแบบการใช้ `np.where`
`np.where(เงื่อนไข, ค่าถ้าจริง, ค่าถ้าเท็จ)`

In [8]:
import pandas as pd
from datetime import datetime
import numpy as np

excel_obj = pd.ExcelFile('mock_isp_data.xlsx')
sheets_name = excel_obj.sheet_names
print(sheets_name)

df = excel_obj.parse('Main_Data')

df['MSISDN'] = df['MSISDN'].apply(str)

date_cols = [
    'CONTRACT_START_DT',
    'CONTRACT_END_DT'
]

df[date_cols] = df[date_cols].apply(pd.to_datetime)

today = pd.Timestamp(datetime.now().date())

# คำนวณวันคงเหลือสัญญา
df['REMAIN_DAYS'] = (df['CONTRACT_END_DT'] - today).dt.days

# มูลค่าสัญญา
df['REMAIN_VALUE'] = df['REMAIN_DAYS'] * (df['RC_RATE'] / 30)


df[['CUST_FULL_NAME', 'REMAIN_DAYS', 'REMAIN_VALUE']]

['Main_Data']


,CUST_FULL_NAME,REMAIN_DAYS,REMAIN_VALUE
0,นายหมดอายุ แล้วครับ,-48,-958.400000
1,บริษัท กำลังจะหมด (15วัน),10,299.666667
2,ร้านค้า ยังเหลือเยอะ,317,12680.000000
3,นายพอดีเป๊ะ (วันนี้หมด),-5,-74.833333
4,บจก. เพิ่งเริ่มสัญญา,682,6797.266667


### 🧬 การแบ่งกลุ่มที่ซับซ้อนขึ้นด้วย `np.select`
บางครั้ง 2 ทางเลือก (จริง/เท็จ) อาจไม่พอสำหรับงาน ISP ของป๋า เช่น ป๋าอยากแบ่งเป็น:
1. **Expired**: วันติดลบ
2. **Warning**: เหลือไม่เกิน 30 วัน
3. **Active**: ยังเหลืออีกเยอะ

เราจะใช้ `np.select` เพื่อทำ **"Mapping"** เงื่อนไขหลาย ๆ ชั้นครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Multi-Condition** | มัล-ติ คอน-ดิ-ชัน | เงื่อนไขหลายชั้น |
| **Mapping** | แมพ-พิง | การจับคู่เงื่อนไขกับผลลัพธ์ที่ต้องการ |
| **Criteria** | ไค-ที-เรีย | เกณฑ์การตัดสินใจ |
| **Default Value** | ดี-ฟอลต์ แว-ลู | ค่าเริ่มต้น (กรณีที่ไม่ตรงกับเงื่อนไขใดเลย) |

In [9]:
# 1. กำหนดรายการเงื่อนไข (Conditions)
conditions = [
    (df['REMAIN_DAYS'] < 0),                          # กลุ่มที่ 1: หมดอายุ
    (df['REMAIN_DAYS'] >= 0) & (df['REMAIN_DAYS'] <= 30), # กลุ่มที่ 2: ใกล้หมด
    (df['REMAIN_DAYS'] > 30)                           # กลุ่มที่ 3: ปกติ
]

# 2. กำหนดชื่อกลุ่ม (Choices) ให้ตรงตามลำดับเงื่อนไข
choices = ['EXPIRED', 'WARNING', 'HEALTHY']

# 3. ใช้ np.select เพื่อสร้างคอลัมน์ 'CONTRACT_STATUS'
# default='UNKNOWN' คือถ้าหลุดจากทุกเงื่อนไขให้ใส่คำนี้
df['CONTRACT_STATUS'] = np.select(conditions, choices, default='UNKNOWN')

# 4. แสดงผลลัพธ์แบบเรียงลำดับสถานะ
display(df[['CUST_FULL_NAME', 'REMAIN_DAYS', 'CONTRACT_STATUS']].sort_values('REMAIN_DAYS'))

,CUST_FULL_NAME,REMAIN_DAYS,CONTRACT_STATUS
0,นายหมดอายุ แล้วครับ,-48,EXPIRED
3,นายพอดีเป๊ะ (วันนี้หมด),-5,EXPIRED
1,บริษัท กำลังจะหมด (15วัน),10,WARNING
2,ร้านค้า ยังเหลือเยอะ,317,HEALTHY
4,บจก. เพิ่งเริ่มสัญญา,682,HEALTHY
